# Quadratic Variation Completion (QVC)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def conv(vec, kernel_size):
    n = vec.shape[0]
    mat = np.zeros((n, kernel_size))
    mat[:, 0] = vec
    for k in range(1, kernel_size):
        mat[:, k] = np.append(vec[n - k :], vec[: n - k], axis = 0)
    return mat

def update_x(z, w, L_tilde, lmbda, gamma):
    x = (lmbda * z - w) @ np.linalg.inv(gamma * (L_tilde + L_tilde.T) / 2 
                                        + lmbda * np.eye(z.shape[1]))
    return x

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def QVC(y_true, y, lmbda, gamma, maxiter = 50):
    eta = 100 * lmbda
    N, T, L = y.shape
    pos_train = np.where(y > 0)
    y_train = y[pos_train]
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    L_tilde = conv(laplacian(T, 1), T)
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = np.zeros((N, T, L))
        for t in range(L):
            x[:, :, t] = update_x(z[:, :, t], w[:, :, t], L_tilde, lmbda, gamma)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

## HighD Dataset

Hyperparameters:

- $\lambda=10^{-3}T$
- $\gamma=5\lambda$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_46_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * T
gamma = 5 * lmbda
maxiter = 100
tensor_hat = QVC(dense_tensor, sparse_tensor, lmbda, gamma, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'HighD_speed_field_70_QVC_rec_lane{}.png'.format(lane + 1))

20
0.09093542496756093
4.533238395102625

40
0.08396313150554544
4.259919737058402

60
0.08255702825421311
4.155761271743132

80
0.08178156510288054
4.087160133462116

100
0.08121844635107761
4.034727495736222

Running time: 5 seconds.


## CitySim Dataset

Hyperparameters:

- $\lambda=10^{-3}T$
- $\gamma=5\lambda$


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * T
gamma = 5 * lmbda
maxiter = 100
tensor_hat = QVC(dense_tensor, sparse_tensor, lmbda, gamma, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'CitySim_speed_field_70_QVC_rec_lane{}.png'.format(lane + 1))

20
0.10549570764309929
3.7692386353728566

40
0.10332548975953246
3.698858200832789

60
0.10330970167555277
3.696847067160917

80
0.10331582291409201
3.696813927859082

100
0.10331747848261702
3.6968269363695465

Running time: 2 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>